In [10]:
from squeezenet import FireModule

In [11]:
# Import necessary libraries
import torch
from squeezenet import FireModule

# Define model parameters
in_channels = 64
squeeze_channels = 16
expansionX1_channels = 64
expansionX3_channels = 64
batch_size = 32
height, width = 128, 128

# Create a random input tensor
x = torch.randn(batch_size, in_channels, height, width)

# Initialize the FireModule
fire_module = FireModule(in_channels, squeeze_channels, expansionX1_channels, expansionX3_channels)

# Forward pass
output = fire_module(x)

# Check the output shape
print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")

AttributeError: cannot assign module before Module.__init__() call